In [1]:
import glob as glob
import os
import json
from llama_index.core import SimpleDirectoryReader
import ollama

from dotenv import load_dotenv
load_dotenv('.env.local') 

True

In [2]:
# ollama model list
from datetime import datetime

# Assuming the ListResponse and related classes are defined as follows:
class ModelDetails:
    def __init__(self, parent_model, format, family, families, parameter_size, quantization_level):
        self.parent_model = parent_model
        self.format = format
        self.family = family
        self.families = families
        self.parameter_size = parameter_size
        self.quantization_level = quantization_level

class Model:
    def __init__(self, model, modified_at, digest, size, details):
        self.model = model
        self.modified_at = modified_at
        self.digest = digest
        self.size = size
        self.details = details

class ListResponse:
    def __init__(self, models):
        self.models = models

In [3]:
ollama.list()

ListResponse(models=[Model(model='smollm2:135m', modified_at=datetime.datetime(2024, 12, 18, 11, 49, 6, 802093, tzinfo=TzInfo(-05:00)), digest='9077fe9d2ae1a4a41a868836b56b8163731a8fe16621397028c2c76f838c6907', size=270898672, details=ModelDetails(parent_model='', format='gguf', family='llama', families=['llama'], parameter_size='134.52M', quantization_level='F16')), Model(model='llava:latest', modified_at=datetime.datetime(2024, 12, 7, 23, 46, 35, 567979, tzinfo=TzInfo(-05:00)), digest='8dd30f6b0cb19f555f2c7a7ebda861449ea2cc76bf1f44e262931f45fc81d081', size=4733363377, details=ModelDetails(parent_model='', format='gguf', family='llama', families=['llama', 'clip'], parameter_size='7B', quantization_level='Q4_0')), Model(model='moondream:latest', modified_at=datetime.datetime(2024, 12, 7, 22, 19, 46, 414440, tzinfo=TzInfo(-05:00)), digest='55fc3abd386771e5b5d1bbcc732f3c3f4df6e9f9f08f1131f9cc27ba2d1eec5b', size=1738451197, details=ModelDetails(parent_model='', format='gguf', family='phi2

In [4]:
ollama_models = ollama.list()

for model in ollama_models.models:
    print(f"Model Name: {model.model}")
    print(f"Modified At: {model.modified_at}")
    print(f"Digest: {model.digest}")
    print(f"Size: {model.size} bytes")
    print(f"Parameter Size: {model.details.parameter_size}")
    print(f"Quantization Level: {model.details.quantization_level}")
    print(f"Family: {model.details.family}")
    print(f"Families: {', '.join(model.details.families)}")
    print("-" * 40)

Model Name: smollm2:135m
Modified At: 2024-12-18 11:49:06.802093-05:00
Digest: 9077fe9d2ae1a4a41a868836b56b8163731a8fe16621397028c2c76f838c6907
Size: 270898672 bytes
Parameter Size: 134.52M
Quantization Level: F16
Family: llama
Families: llama
----------------------------------------
Model Name: llava:latest
Modified At: 2024-12-07 23:46:35.567979-05:00
Digest: 8dd30f6b0cb19f555f2c7a7ebda861449ea2cc76bf1f44e262931f45fc81d081
Size: 4733363377 bytes
Parameter Size: 7B
Quantization Level: Q4_0
Family: llama
Families: llama, clip
----------------------------------------
Model Name: moondream:latest
Modified At: 2024-12-07 22:19:46.414440-05:00
Digest: 55fc3abd386771e5b5d1bbcc732f3c3f4df6e9f9f08f1131f9cc27ba2d1eec5b
Size: 1738451197 bytes
Parameter Size: 1B
Quantization Level: Q4_0
Family: phi2
Families: phi2, clip
----------------------------------------
Model Name: llama3.2:latest
Modified At: 2024-12-07 22:18:36.845215-05:00
Digest: a80c4f17acd55265feec403c7aef86be0c25983ab279d83f3bcd3ab

In [20]:
import json
import os
import time
from litellm import completion, success_callback
from llama_index.core import SimpleDirectoryReader
from pprint import pprint

# Global cost tracker
COST_TRACKER = {"cost": 0.0}

# track_cost_callback
def track_cost_callback(kwargs, completion_response, start_time, end_time, stream=False):
    # Extract usage from the response if available and calculate cost.
    # This calculation is just an example; the actual cost logic depends on your provider/pricing model.
    try:
        # Try converting the response to a dict
        if hasattr(completion_response, "to_dict"):
            response_dict = completion_response.to_dict()
        else:
            response_dict = json.loads(str(completion_response))
        
        usage = response_dict.get("usage", {})
        # Example: If each 1K tokens cost $0.003:
        total_tokens = usage.get("total_tokens", 0)
        COST_TRACKER["cost"] = (total_tokens / 1000.0) * 0.003
        
        print("Calculated cost:", COST_TRACKER["cost"])
    except Exception as e:
        print("Error in track_cost_callback:", e)

# Set the success callback
success_callback[:] = [track_cost_callback]

#######################################################
# def create_file_tree(
#     summaries: list,
#     host: str,
#     port: int,
#     model: str = "llama-3.1-70b-versatile",
#     api_base: str = None,
#     stream: bool = False
# ) -> list:
#     """
#     Create a file tree based on the provided summaries.

#     Args:
#         summaries (list): List of file summaries to organize.
#         host (str): Host address for the API.
#         port (int): Port number for the API.
#         model (str, optional): AI model to use for file organization. 
#                                Defaults to "llama-3.1-70b-versatile".
#         api_base (str, optional): Base URL for the API. If not provided, constructed from host and port.
#         stream (bool, optional): Whether to use streaming. Defaults to False.

#     Returns:
#         list: Reorganized file paths.
#     """
#     PROMPT = """
#     You will be provided with a list of source files and a summary of their contents. For each file, propose a new path and filename, using a directory structure that optimally organizes the files using known conventions and best practices.
#     Follow good naming conventions. Here are a few guidelines:
#     - Think about your files: What related files are you working with?
#     - Identify metadata (for example, date, sample, experiment): What information is needed to easily locate a specific file?
#     - Abbreviate or encode metadata
#     - Use versioning: Are you maintaining different versions of the same file?
#     - Think about how you will search for your files: What comes first?
#     - Deliberately separate metadata elements: Avoid spaces or special characters in your file names
#     If the file is already named well or matches a known convention, set the destination path to the same as the source path.

#     Your response must be a JSON object with the following schema:
#     ```json
#     {
#         "files": [
#             {
#                 "src_path": "original file path",
#                 "dst_path": "new file path under proposed directory structure with proposed file name"
#             }
#         ]
#     }
#     ```
#     """.strip()

#     if not api_base:
#         api_base = f"http://{host}:{port}"

#     try:
#         response = completion(
#             model=model,
#             messages=[
#                 {"role": "system", "content": PROMPT},
#                 {"role": "user", "content": json.dumps(summaries)},
#             ],
#             response_format={"type": "json_object"},
#             temperature=0,
#             api_base=api_base,
#             stream=stream
#         )
#     except Exception as e:
#         print(f"LiteLLM Error >>> {e}")
#         return []

#     if response is None:
#         print("No response received from the API.")
#         return []

#     # Convert the response to a dict
#     try:
#         if hasattr(response, "to_dict"):
#             response_dict = response.to_dict()
#         else:
#             response_dict = json.loads(str(response))
#     except (TypeError, json.JSONDecodeError) as e:
#         print(f"Error parsing response: {e}")
#         return []

#     content = response_dict.get("choices", [{}])[0].get("message", {}).get("content", "")

#     # Attempt to parse the JSON content
#     try:
#         file_tree = json.loads(content)["files"]
#     except json.JSONDecodeError as e:
#         print(f"Error decoding JSON content: {e}")
#         return []
#     except KeyError:
#         print("The expected 'files' key is missing in the response.")
#         return []

#     # Optional: Pretty-print the file tree for debugging
#     # print("File Tree: >>>>>>>>>>> ")
#     # pprint(file_tree)

#     return file_tree

#######################################################
import json
from typing import List, Dict, Any
from pprint import pprint

def create_file_tree(
    summaries: List[Dict[str, Any]],
    host: str,
    port: int,
    model: str = "llama-3.1-70b-versatile",
    api_base: str = None,
    stream: bool = False
) -> List[Dict[str, str]]:
    """
    Create a file tree based on the provided summaries.

    Args:
        summaries (list): List of file summaries to organize.
        host (str): Host address for the API.
        port (int): Port number for the API.
        model (str, optional): AI model to use for file organization. 
                               Defaults to "llama-3.1-70b-versatile".
        api_base (str, optional): Base URL for the API. If not provided, constructed from host and port.
        stream (bool, optional): Whether to use streaming. Defaults to False.

    Returns:
        list: Reorganized file paths.
    """
    PROMPT = """
    You will be provided with a list of source files and a summary of their contents. For each file, propose a new path and filename, using a directory structure that optimally organizes the files using known conventions and best practices.
    Follow good naming conventions. Here are a few guidelines:
    - Think about your files: What related files are you working with?
    - Identify metadata (for example, date, sample, experiment): What information is needed to easily locate a specific file?
    - Abbreviate or encode metadata
    - Use versioning: Are you maintaining different versions of the same file?
    - Think about how you will search for your files: What comes first?
    - Deliberately separate metadata elements: Avoid spaces or special characters in your file names
    - Do not change the file extension
    If the file is already named well or matches a known convention, set the destination path to the same as the source path.

    Your response must be a JSON object with the following schema:
    ```json
    {
        "files": [
            {
                "src_path": "original file path",
                "dst_path": "new file path under proposed directory structure with proposed file name"
            }
        ]
    }
    ```
    """.strip()

    if not api_base:
        api_base = f"http://{host}:{port}"
        print(">>> api_base: ", api_base)

    try:
        response = completion(
            model=model,
            messages=[
                {"role": "system", "content": PROMPT},
                {"role": "user", "content": json.dumps(summaries)},
            ],
            response_format={"type": "json_object"},
            # temperature=0,
            api_base=api_base,
            stream=stream
        )
    except Exception as e:
        print(f"LiteLLM Error >>> {e}")
        return []

    if response is None:
        print("No response received from the API.")
        return []

    # Convert the response to a dict
    try:
        if hasattr(response, "to_dict"):
            response_dict = response.to_dict()
        else:
            response_dict = json.loads(str(response))
    except (TypeError, json.JSONDecodeError) as e:
        print(f"Error parsing response: {e}")
        return []

    content = response_dict.get("choices", [{}])[0].get("message", {}).get("content", "")

    # Attempt to parse the JSON content
    try:
        file_tree = json.loads(content)["files"]
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON content: {e}")
        return []
    except KeyError:
        print("The expected 'files' key is missing in the response.")
        return []

    return file_tree

#######################################################
# import json
# from typing import List, Dict, Any
# from pprint import pprint
# from litellm import completion  # Ensure you have litellm imported correctly

# def create_file_tree(
#     summaries: List[Dict[str, Any]],
#     host: str,
#     port: int,
#     model: str = "llama-3.1-70b-versatile",
#     api_base: str = None,
#     stream: bool = False
# ) -> List[Dict[str, str]]:
#     """
#     Create a file tree based on the provided summaries.

#     Args:
#         summaries (list): List of file summaries to organize.
#         host (str): Host address for the API.
#         port (int): Port number for the API.
#         model (str, optional): AI model to use for file organization. 
#                                Defaults to "llama-3.1-70b-versatile".
#         api_base (str, optional): Base URL for the API. If not provided, constructed from host and port.
#         stream (bool, optional): Whether to use streaming. Defaults to False.

#     Returns:
#         list: Reorganized file paths.
#     """
#     PROMPT = """
#     You will be provided with a list of source files and a summary of their contents. For each file, propose a new path and filename, using a directory structure that optimally organizes the files using known conventions and best practices.
#     Follow good naming conventions. Here are a few guidelines:
#     - Think about your files: What related files are you working with?
#     - Identify metadata (for example, date, sample, experiment): What information is needed to easily locate a specific file?
#     - Abbreviate or encode metadata
#     - Use versioning: Are you maintaining different versions of the same file?
#     - Think about how you will search for your files: What comes first?
#     - Deliberately separate metadata elements: Avoid spaces or special characters in your file names
#     If the file is already named well or matches a known convention, set the destination path to the same as the source path.

#     Your response must be a JSON object with the following schema:
#     ```json
#     {
#         "files": [
#             {
#                 "src_path": "original file path",
#                 "dst_path": "new file path under proposed directory structure with proposed file name"
#             }
#         ]
#     }
#     ```
#     """.strip()

#     if not api_base:
#         api_base = f"http://{host}:{port}"
#         print(">>> api_base: ", api_base)

#     try:
#         response = completion(
#             model=model,
#             messages=[
#                 {"role": "system", "content": PROMPT},
#                 {"role": "user", "content": json.dumps(summaries)},
#             ],
#             response_format="json",  # Set to 'json' to receive a JSON object
#             functions=None,          # Disable function calling
#             api_base=api_base,
#             stream=stream
#         )
#     except Exception as e:
#         print(f"LiteLLM Error >>> {e}")
#         return []

#     if response is None:
#         print("No response received from the API.")
#         return []

#     # Convert the response to a dict
#     try:
#         if hasattr(response, "to_dict"):
#             response_dict = response.to_dict()
#         else:
#             response_dict = json.loads(str(response))
#     except (TypeError, json.JSONDecodeError) as e:
#         print(f"Error parsing response: {e}")
#         return []

#     content = response_dict.get("choices", [{}])[0].get("message", {}).get("content", "")

#     # Attempt to parse the JSON content
#     try:
#         file_tree = json.loads(content)["files"]
#     except json.JSONDecodeError as e:
#         print(f"Error decoding JSON content: {e}")
#         return []
#     except KeyError:
#         print("The expected 'files' key is missing in the response.")
#         return []

#     return file_tree

#####################################################
def load_documents(path: str) -> list:
    """
    Load documents from the specified path.

    Args:
        path (str): Directory path to load documents from.

    Returns:
        list: List of document dictionaries containing content and metadata.
    """
    reader = SimpleDirectoryReader(input_dir=path)
    documents = reader.load_data()
    doc_dicts = [{"content": d.text, **d.metadata} for d in documents]
    return doc_dicts

def process_metadata(doc_dicts: list) -> list:
    """
    Process metadata to remove duplicate file entries.

    Args:
        doc_dicts (list): List of document dictionaries.

    Returns:
        list: List of unique document dictionaries based on file path.
    """
    file_seen = set()
    metadata_list = []
    for doc in doc_dicts:
        if doc["file_path"] not in file_seen:
            file_seen.add(doc["file_path"])
            metadata_list.append(doc)
    return metadata_list

def query_summaries(
    doc_dicts: list,
    host: str,
    port: int,
    model: str,
    api_base: str = None,
    stream: bool = False
) -> dict:
    """
    Generate summaries for the provided document dictionaries.

    Args:
        doc_dicts (list): List of document dictionaries.
        host (str): Host address for the API.
        port (int): Port number for the API.
        model (str): AI model to use for generating summaries.
        api_base (str, optional): Base URL for the API. Defaults to None.
        stream (bool, optional): Whether to use streaming. Defaults to False.

    Returns:
        dict: Dictionary containing file summaries and associated metadata.
    """
    PROMPT = f""" 
    The following is a list of file contents, along with their metadata. For each file, provide a summary of the contents. The purpose of the summary is to organize files based on their content. To this end provide a concise but informative summary. Try to make the summary as specific to the file as possible. {doc_dicts}
    
    Return a JSON object with the following schema:
    
json
    {{
      "files": [
        {{
          "file_path": "path to the file including name",
          "summary": "summary of the content"
        }}
      ]
    }}
    """.strip()

    if not api_base:
        api_base = f"http://{host}:{port}"

    response = None
    try:
        response = completion(
            model=model, 
            messages=[
                {
                    "role": "system", 
                    "content": "Always return JSON. Do not include any other text or formatting characters."
                },
                {
                    "role": "user", 
                    "content": PROMPT
                }
            ],
            api_base=api_base,
            stream=stream
        )
    except Exception as e:
        print("LiteLLM Error >>> ", e)
        return {"files": [], "cost": COST_TRACKER["cost"]}

    if response is None:
        return {"files": [], "cost": COST_TRACKER["cost"]}

    # Convert the response to a dict
    if hasattr(response, "to_dict"):
        response_dict = response.to_dict()
    else:
        try:
            response_dict = json.loads(str(response))
        except (TypeError, json.JSONDecodeError):
            return {"files": [], "cost": COST_TRACKER["cost"]}

    content = response_dict.get("choices", [{}])[0].get("message", {}).get("content", "")

    # Attempt to parse the JSON content
    try:
        summaries = json.loads(content)
    except json.JSONDecodeError:
        summaries = {"files": []}

    print("Summaries: >>>>>>>>>>> ")
    pprint(summaries)

    # If summaries is a list and we expect a dict, take the first element
    if isinstance(summaries, list) and len(summaries) > 0 and isinstance(summaries[0], dict):
        summaries = summaries[0]

    # Add usage tokens if available
    usage = response_dict.get("usage", {})

    # print("Usage: >>>>>>>>>>> ")
    print("Usage: >>>>>>>>>>> ", usage)

    if usage and isinstance(summaries, dict):
        summaries["usage"] = {
            "completion_tokens": usage.get("completion_tokens"),
            "prompt_tokens": usage.get("prompt_tokens"),
            "total_tokens": usage.get("total_tokens")
        }

    # Add cost from the COST_TRACKER
    if isinstance(summaries, dict):
        summaries["cost"] = COST_TRACKER["cost"]
    else:
        # If still not dict, fallback to a dict structure
        summaries = {
            "files": [],
            "cost": COST_TRACKER["cost"]
        }

    return summaries

In [21]:
path = "/home/rakesh/Downloads/sample_data/four/"
doc_dicts = load_documents(path)
print("# of docs: ", len(doc_dicts))  # doc_dicts
pprint(doc_dicts)

# of docs:  3
[{'content': 'In the ever-evolving landscape of artificial intelligence, '
             'Vertical Large Language Model (LLM) Agents are emerging as the '
             'next big wave in Software as a Service (SaaS) innovations. '
             'Imagine having an AI assistant tailored specifically to your '
             'industry’s unique challenges and workflows, capable of '
             'performing complex tasks in a fraction of the time it '
             'traditionally takes. This is not a distant future but a present '
             'reality, as evidenced by the remarkable success stories like '
             'Case Text’s Co-Counsel. But why are these vertical LLM agents '
             'poised to become billion-dollar opportunities, and how can your '
             'organization leverage this transformative technology?\n'
             '\n'
             'The Moonshot Moment: A Leap Ahead of the Market\n'
             'Jake Heller, founder of Case Text, aptly describes their

# Ollama

In [29]:
# Example: Using Ollama llama model
summaries_ol = query_summaries(
    doc_dicts=doc_dicts,
    host="localhost",
    port="11434",
    model="ollama/llama3.2:latest",
    # model="ollama/smollm2:135m",
    api_base="http://localhost:11434"
)

pprint(summaries_ol)


Summaries: >>>>>>>>>>> 
[{'file_path': '/home/rakesh/Downloads/sample_data/four/dsflsdflj.txt',
  'summary': 'An article discussing Vertical Large Language Model (LLM) Agents '
             'and their potential to revolutionize industries, using the '
             'success story of Co-Counsel as an example.'},
 {'file_path': '/home/rakesh/Downloads/sample_data/four/random_file.txt',
  'summary': 'A test file containing bank account information'},
 {'file_path': '/home/rakesh/Downloads/sample_data/four/shad.txt',
  'summary': 'An article introducing the Next.js Starter Kit, a comprehensive '
             'template for building SAAS products with Next.js'}]
Usage: >>>>>>>>>>>  {'completion_tokens': 155, 'prompt_tokens': 1431, 'total_tokens': 1586, 'completion_tokens_details': None, 'prompt_tokens_details': None}
{'cost': 0.003762,
 'file_path': '/home/rakesh/Downloads/sample_data/four/dsflsdflj.txt',
 'summary': 'An article discussing Vertical Large Language Model (LLM) Agents '
        

Calculated cost: 0.004758


In [30]:
pprint(summaries_ol)

{'cost': 0.003762,
 'file_path': '/home/rakesh/Downloads/sample_data/four/dsflsdflj.txt',
 'summary': 'An article discussing Vertical Large Language Model (LLM) Agents '
            'and their potential to revolutionize industries, using the '
            'success story of Co-Counsel as an example.',
 'usage': {'completion_tokens': 155,
           'prompt_tokens': 1431,
           'total_tokens': 1586}}


In [ ]:
# summaries_ol1 = [{'file_path': '/home/rakesh/Downloads/sample_data/four/dsflsdflj.txt',
#   'summary': 'This file discusses the emergence of Vertical Large Language '
#              'Model (LLM) Agents as the next big wave in Software as a Service '
#              '(SaaS) innovations, highlighting the success story of Case '
#              "Text's Co-Counsel, an AI-driven legal assistant that "
#              "dramatically increased the company's valuation."},
#  {'file_path': '/home/rakesh/Downloads/sample_data/four/random_file.txt',
#   'summary': 'This file contains some generic test information, including a '
#              'bank account number.'},
#  {'file_path': '/home/rakesh/Downloads/sample_data/four/shad.txt',
#   'summary': 'This file provides an overview of several open-source Next.js '
#              'starter projects, including their tech stacks, features, and use '
#              'cases, such as blogging, admin dashboards, and AI-powered '
#              'applications.'}]


tree_ol = create_file_tree(
    summaries=summaries_ol,  # Pass the list of files
    host="localhost",
    port=11434,  # Pass port as an integer
    model="ollama/llama3.2:latest",
    # model="ollama/smollm2:135m",
    api_base="http://localhost:11434"  # Ensure this matches your API's expected base URL
)

pprint(tree_ol)


In [ ]:

if __name__ == "__main__":
    # Define your parameters
    source_directory = "/home/rakesh/Downloads/sample_data/four/"  # Replace with your actual source documents path
    destination_directory = "/home/rakesh/Downloads/sample_data/four_organized/"  # Replace with your desired destination path
    api_host = "localhost"                                         # API host address (not needed if api_base is provided)
    api_port = 8111                                                # API port number
    summary_model = "ollama/llama3.2:latest"
    tree_model = "ollama/llama3.2:latest"
    api_base_url = "http://localhost:11434"                     # Base URL for the API
    use_streaming = False                                          # Set to True if you want streaming
    # model="ollama/smollm2:135m",

    # Ensure destination directory exists
    os.makedirs(destination_directory, exist_ok=True)

    # Run the workflow
    organized_files_result = document_processing_workflow(
        source_path=source_directory,
        destination_path=destination_directory,
        api_host=api_host,
        api_port=api_port,
        summary_model=summary_model,
        tree_model=tree_model,
        api_base=api_base_url,                                  # Providing the API base URL
        stream=use_streaming
    )

    # Extract the results
    summaries = organized_files_result.get("summaries", {})
    file_tree = organized_files_result.get("file_tree", [])
    concatenated_data = organized_files_result.get("concatenated_data", [])

    # Optionally, pretty-print the results using Rich
    console.print("[bold yellow]Summaries:[/bold yellow]")
    pprint(json.dumps(summaries, indent=4))

    console.print("\n[bold yellow]File Tree:[/bold yellow]")
    pprint(json.dumps(file_tree, indent=4))

    console.print("\n[bold yellow]Concatenated Data:[/bold yellow]")
    pprint(json.dumps(concatenated_data, indent=4))


15:53:53.355 | INFO    | prefect.engine - Created flow run 'ochre-sidewinder' for flow 'Document Processing Workflow'

15:53:53.411 | INFO    | prefect - Success callback set successfully.

15:53:53.413 | INFO    | Task run 'set_success_callback-659' - Finished in state Completed()

                                              Available Ollama Models                                              
                   ╷              ╷              ╷              ╷            ╷             ╷        ╷              
  Model Name       │ Modified At  │ Digest       │ Size (bytes) │ Param Size │ Quant Level │ Family │ Families     
 ══════════════════╪══════════════╪══════════════╪══════════════╪════════════╪═════════════╪════════╪═════════════ 
  smollm2:135m     │ 2024-12-18   │ 9077fe9d2ae… │    270898672 │    134.52M │ F16         │ llama  │ llama        
                   │ 11:49:06.80… │              │              │            │             │        │              
  llava:latest     │ 2024-12-07   │ 8dd30f6b0cb… │   4733363377 │         7B │ Q4_0        │ llama  │ llama, clip  
                   │ 23:46:35.56… │              │              │            │             │        │              
  moondream:latest │ 2024-12-07   │ 55fc3abd386… │   1738451197 │         1B │ Q4_0        │ phi2   │ phi2, clip   
                   │ 22:19:46.41… │              │              │            │             │        │              
  llama3.2:latest  │ 2024-12-07   │ a80c4f17acd… │   2019393189 │       3.2B │ Q4_K_M      │ llama  │ llama        
                   │ 22:18:36.84… │              │              │            │             │        │              
                   ╵              ╵              ╵              ╵            ╵             ╵        ╵

15:53:53.446 | INFO    | Task run 'list_ollama_models-a28' - Finished in state Completed()

...

('{\n'
 '    "files": [\n'
 '        {\n'
 '            "file_path": '
 '"/home/rakesh/Downloads/sample_data/four/dsflsdflj.txt",\n'
 '            "summary": "Introduction to Vertical Large Language Model (LLM) '
 'Agents and their applications in Software as a Service (SaaS) innovations, '
 'including the success story of Case Text\'s Co-Counsel."\n'
 '        },\n'
 '        {\n'
 '            "file_path": '
 '"/home/rakesh/Downloads/sample_data/four/random_file.txt",\n'
 '            "summary": "A test file containing important information, such '
 'as a bank account number."\n'
 '        },\n'
 '        {\n'
 '            "file_path": '
 '"/home/rakesh/Downloads/sample_data/four/shad.txt",\n'
 '            "summary": "Information about Next.js, including its features, '
 'technologies used, and recommended boilerplates for development."\n'
 '        }\n'
 '    ],\n'
 '    "usage": {\n'
 '        "completion_tokens": 139,\n'
 '        "prompt_tokens": 1436,\n'
 '        "total_token

File Tree:

'[]'


Concatenated Data:

'[]'


In [ ]:

if __name__ == "__main__":
    # Define your parameters
    source_directory = "/home/rakesh/Downloads/sample_data/four/"  # Replace with your actual source documents path
    destination_directory = "/home/rakesh/Downloads/sample_data/four_organized/"  # Replace with your desired destination path
    api_host = "localhost"                                         # API host address (not needed if api_base is provided)
    api_port = 8111                                                # API port number
    summary_model = "ollama/llama3.2:latest"
    tree_model = "ollama/llama3.2:latest"
    api_base_url = "http://localhost:11434"                     # Base URL for the API
    use_streaming = False                                          # Set to True if you want streaming
    # model="ollama/smollm2:135m",

    # Ensure destination directory exists
    os.makedirs(destination_directory, exist_ok=True)

    # Run the workflow
    organized_files_result = document_processing_workflow(
        source_path=source_directory,
        destination_path=destination_directory,
        api_host=api_host,
        api_port=api_port,
        summary_model=summary_model,
        tree_model=tree_model,
        api_base=api_base_url,                                  # Providing the API base URL
        stream=use_streaming
    )

    # Extract the results
    summaries = organized_files_result.get("summaries", {})
    file_tree = organized_files_result.get("file_tree", [])
    concatenated_data = organized_files_result.get("concatenated_data", [])

    # Optionally, pretty-print the results using Rich
    console.print("[bold yellow]Summaries:[/bold yellow]")
    pprint(json.dumps(summaries, indent=4))

    console.print("\n[bold yellow]File Tree:[/bold yellow]")
    pprint(json.dumps(file_tree, indent=4))

    console.print("\n[bold yellow]Concatenated Data:[/bold yellow]")
    pprint(json.dumps(concatenated_data, indent=4))


15:53:53.355 | INFO    | prefect.engine - Created flow run 'ochre-sidewinder' for flow 'Document Processing Workflow'

15:53:53.411 | INFO    | prefect - Success callback set successfully.

15:53:53.413 | INFO    | Task run 'set_success_callback-659' - Finished in state Completed()

                                              Available Ollama Models                                              
                   ╷              ╷              ╷              ╷            ╷             ╷        ╷              
  Model Name       │ Modified At  │ Digest       │ Size (bytes) │ Param Size │ Quant Level │ Family │ Families     
 ══════════════════╪══════════════╪══════════════╪══════════════╪════════════╪═════════════╪════════╪═════════════ 
  smollm2:135m     │ 2024-12-18   │ 9077fe9d2ae… │    270898672 │    134.52M │ F16         │ llama  │ llama        
                   │ 11:49:06.80… │              │              │            │             │        │              
  llava:latest     │ 2024-12-07   │ 8dd30f6b0cb… │   4733363377 │         7B │ Q4_0        │ llama  │ llama, clip  
                   │ 23:46:35.56… │              │              │            │             │        │              
  moondream:latest │ 2024-12-07   │ 55fc3abd386… │   1738451197 │         1B │ Q4_0        │ phi2   │ phi2, clip   
                   │ 22:19:46.41… │              │              │            │             │        │              
  llama3.2:latest  │ 2024-12-07   │ a80c4f17acd… │   2019393189 │       3.2B │ Q4_K_M      │ llama  │ llama        
                   │ 22:18:36.84… │              │              │            │             │        │              
                   ╵              ╵              ╵              ╵            ╵             ╵        ╵

15:53:53.446 | INFO    | Task run 'list_ollama_models-a28' - Finished in state Completed()

...

('{\n'
 '    "files": [\n'
 '        {\n'
 '            "file_path": '
 '"/home/rakesh/Downloads/sample_data/four/dsflsdflj.txt",\n'
 '            "summary": "Introduction to Vertical Large Language Model (LLM) '
 'Agents and their applications in Software as a Service (SaaS) innovations, '
 'including the success story of Case Text\'s Co-Counsel."\n'
 '        },\n'
 '        {\n'
 '            "file_path": '
 '"/home/rakesh/Downloads/sample_data/four/random_file.txt",\n'
 '            "summary": "A test file containing important information, such '
 'as a bank account number."\n'
 '        },\n'
 '        {\n'
 '            "file_path": '
 '"/home/rakesh/Downloads/sample_data/four/shad.txt",\n'
 '            "summary": "Information about Next.js, including its features, '
 'technologies used, and recommended boilerplates for development."\n'
 '        }\n'
 '    ],\n'
 '    "usage": {\n'
 '        "completion_tokens": 139,\n'
 '        "prompt_tokens": 1436,\n'
 '        "total_token

File Tree:

'[]'


Concatenated Data:

'[]'


In [ ]:

if __name__ == "__main__":
    # Define your parameters
    source_directory = "/home/rakesh/Downloads/sample_data/four/"  # Replace with your actual source documents path
    destination_directory = "/home/rakesh/Downloads/sample_data/four_organized/"  # Replace with your desired destination path
    api_host = "localhost"                                         # API host address (not needed if api_base is provided)
    api_port = 8111                                                # API port number
    summary_model = "ollama/llama3.2:latest"
    tree_model = "ollama/llama3.2:latest"
    api_base_url = "http://localhost:11434"                     # Base URL for the API
    use_streaming = False                                          # Set to True if you want streaming
    # model="ollama/smollm2:135m",

    # Ensure destination directory exists
    os.makedirs(destination_directory, exist_ok=True)

    # Run the workflow
    organized_files_result = document_processing_workflow(
        source_path=source_directory,
        destination_path=destination_directory,
        api_host=api_host,
        api_port=api_port,
        summary_model=summary_model,
        tree_model=tree_model,
        api_base=api_base_url,                                  # Providing the API base URL
        stream=use_streaming
    )

    # Extract the results
    summaries = organized_files_result.get("summaries", {})
    file_tree = organized_files_result.get("file_tree", [])
    concatenated_data = organized_files_result.get("concatenated_data", [])

    # Optionally, pretty-print the results using Rich
    console.print("[bold yellow]Summaries:[/bold yellow]")
    pprint(json.dumps(summaries, indent=4))

    console.print("\n[bold yellow]File Tree:[/bold yellow]")
    pprint(json.dumps(file_tree, indent=4))

    console.print("\n[bold yellow]Concatenated Data:[/bold yellow]")
    pprint(json.dumps(concatenated_data, indent=4))


15:53:53.355 | INFO    | prefect.engine - Created flow run 'ochre-sidewinder' for flow 'Document Processing Workflow'

15:53:53.411 | INFO    | prefect - Success callback set successfully.

15:53:53.413 | INFO    | Task run 'set_success_callback-659' - Finished in state Completed()

                                              Available Ollama Models                                              
                   ╷              ╷              ╷              ╷            ╷             ╷        ╷              
  Model Name       │ Modified At  │ Digest       │ Size (bytes) │ Param Size │ Quant Level │ Family │ Families     
 ══════════════════╪══════════════╪══════════════╪══════════════╪════════════╪═════════════╪════════╪═════════════ 
  smollm2:135m     │ 2024-12-18   │ 9077fe9d2ae… │    270898672 │    134.52M │ F16         │ llama  │ llama        
                   │ 11:49:06.80… │              │              │            │             │        │              
  llava:latest     │ 2024-12-07   │ 8dd30f6b0cb… │   4733363377 │         7B │ Q4_0        │ llama  │ llama, clip  
                   │ 23:46:35.56… │              │              │            │             │        │              
  moondream:latest │ 2024-12-07   │ 55fc3abd386… │   1738451197 │         1B │ Q4_0        │ phi2   │ phi2, clip   
                   │ 22:19:46.41… │              │              │            │             │        │              
  llama3.2:latest  │ 2024-12-07   │ a80c4f17acd… │   2019393189 │       3.2B │ Q4_K_M      │ llama  │ llama        
                   │ 22:18:36.84… │              │              │            │             │        │              
                   ╵              ╵              ╵              ╵            ╵             ╵        ╵

15:53:53.446 | INFO    | Task run 'list_ollama_models-a28' - Finished in state Completed()

...

('{\n'
 '    "files": [\n'
 '        {\n'
 '            "file_path": '
 '"/home/rakesh/Downloads/sample_data/four/dsflsdflj.txt",\n'
 '            "summary": "Introduction to Vertical Large Language Model (LLM) '
 'Agents and their applications in Software as a Service (SaaS) innovations, '
 'including the success story of Case Text\'s Co-Counsel."\n'
 '        },\n'
 '        {\n'
 '            "file_path": '
 '"/home/rakesh/Downloads/sample_data/four/random_file.txt",\n'
 '            "summary": "A test file containing important information, such '
 'as a bank account number."\n'
 '        },\n'
 '        {\n'
 '            "file_path": '
 '"/home/rakesh/Downloads/sample_data/four/shad.txt",\n'
 '            "summary": "Information about Next.js, including its features, '
 'technologies used, and recommended boilerplates for development."\n'
 '        }\n'
 '    ],\n'
 '    "usage": {\n'
 '        "completion_tokens": 139,\n'
 '        "prompt_tokens": 1436,\n'
 '        "total_token

File Tree:

'[]'


Concatenated Data:

'[]'


In [ ]:

if __name__ == "__main__":
    # Define your parameters
    source_directory = "/home/rakesh/Downloads/sample_data/four/"  # Replace with your actual source documents path
    destination_directory = "/home/rakesh/Downloads/sample_data/four_organized/"  # Replace with your desired destination path
    api_host = "localhost"                                         # API host address (not needed if api_base is provided)
    api_port = 8111                                                # API port number
    summary_model = "ollama/llama3.2:latest"
    tree_model = "ollama/llama3.2:latest"
    api_base_url = "http://localhost:11434"                     # Base URL for the API
    use_streaming = False                                          # Set to True if you want streaming
    # model="ollama/smollm2:135m",

    # Ensure destination directory exists
    os.makedirs(destination_directory, exist_ok=True)

    # Run the workflow
    organized_files_result = document_processing_workflow(
        source_path=source_directory,
        destination_path=destination_directory,
        api_host=api_host,
        api_port=api_port,
        summary_model=summary_model,
        tree_model=tree_model,
        api_base=api_base_url,                                  # Providing the API base URL
        stream=use_streaming
    )

    # Extract the results
    summaries = organized_files_result.get("summaries", {})
    file_tree = organized_files_result.get("file_tree", [])
    concatenated_data = organized_files_result.get("concatenated_data", [])

    # Optionally, pretty-print the results using Rich
    console.print("[bold yellow]Summaries:[/bold yellow]")
    pprint(json.dumps(summaries, indent=4))

    console.print("\n[bold yellow]File Tree:[/bold yellow]")
    pprint(json.dumps(file_tree, indent=4))

    console.print("\n[bold yellow]Concatenated Data:[/bold yellow]")
    pprint(json.dumps(concatenated_data, indent=4))


15:53:53.355 | INFO    | prefect.engine - Created flow run 'ochre-sidewinder' for flow 'Document Processing Workflow'

15:53:53.411 | INFO    | prefect - Success callback set successfully.

15:53:53.413 | INFO    | Task run 'set_success_callback-659' - Finished in state Completed()

                                              Available Ollama Models                                              
                   ╷              ╷              ╷              ╷            ╷             ╷        ╷              
  Model Name       │ Modified At  │ Digest       │ Size (bytes) │ Param Size │ Quant Level │ Family │ Families     
 ══════════════════╪══════════════╪══════════════╪══════════════╪════════════╪═════════════╪════════╪═════════════ 
  smollm2:135m     │ 2024-12-18   │ 9077fe9d2ae… │    270898672 │    134.52M │ F16         │ llama  │ llama        
                   │ 11:49:06.80… │              │              │            │             │        │              
  llava:latest     │ 2024-12-07   │ 8dd30f6b0cb… │   4733363377 │         7B │ Q4_0        │ llama  │ llama, clip  
                   │ 23:46:35.56… │              │              │            │             │        │              
  moondream:latest │ 2024-12-07   │ 55fc3abd386… │   1738451197 │         1B │ Q4_0        │ phi2   │ phi2, clip   
                   │ 22:19:46.41… │              │              │            │             │        │              
  llama3.2:latest  │ 2024-12-07   │ a80c4f17acd… │   2019393189 │       3.2B │ Q4_K_M      │ llama  │ llama        
                   │ 22:18:36.84… │              │              │            │             │        │              
                   ╵              ╵              ╵              ╵            ╵             ╵        ╵

15:53:53.446 | INFO    | Task run 'list_ollama_models-a28' - Finished in state Completed()

...

('{\n'
 '    "files": [\n'
 '        {\n'
 '            "file_path": '
 '"/home/rakesh/Downloads/sample_data/four/dsflsdflj.txt",\n'
 '            "summary": "Introduction to Vertical Large Language Model (LLM) '
 'Agents and their applications in Software as a Service (SaaS) innovations, '
 'including the success story of Case Text\'s Co-Counsel."\n'
 '        },\n'
 '        {\n'
 '            "file_path": '
 '"/home/rakesh/Downloads/sample_data/four/random_file.txt",\n'
 '            "summary": "A test file containing important information, such '
 'as a bank account number."\n'
 '        },\n'
 '        {\n'
 '            "file_path": '
 '"/home/rakesh/Downloads/sample_data/four/shad.txt",\n'
 '            "summary": "Information about Next.js, including its features, '
 'technologies used, and recommended boilerplates for development."\n'
 '        }\n'
 '    ],\n'
 '    "usage": {\n'
 '        "completion_tokens": 139,\n'
 '        "prompt_tokens": 1436,\n'
 '        "total_token

File Tree:

'[]'


Concatenated Data:

'[]'


In [ ]:

if __name__ == "__main__":
    # Define your parameters
    source_directory = "/home/rakesh/Downloads/sample_data/four/"  # Replace with your actual source documents path
    destination_directory = "/home/rakesh/Downloads/sample_data/four_organized/"  # Replace with your desired destination path
    api_host = "localhost"                                         # API host address (not needed if api_base is provided)
    api_port = 8111                                                # API port number
    summary_model = "ollama/llama3.2:latest"
    tree_model = "ollama/llama3.2:latest"
    api_base_url = "http://localhost:11434"                     # Base URL for the API
    use_streaming = False                                          # Set to True if you want streaming
    # model="ollama/smollm2:135m",

    # Ensure destination directory exists
    os.makedirs(destination_directory, exist_ok=True)

    # Run the workflow
    organized_files_result = document_processing_workflow(
        source_path=source_directory,
        destination_path=destination_directory,
        api_host=api_host,
        api_port=api_port,
        summary_model=summary_model,
        tree_model=tree_model,
        api_base=api_base_url,                                  # Providing the API base URL
        stream=use_streaming
    )

    # Extract the results
    summaries = organized_files_result.get("summaries", {})
    file_tree = organized_files_result.get("file_tree", [])
    concatenated_data = organized_files_result.get("concatenated_data", [])

    # Optionally, pretty-print the results using Rich
    console.print("[bold yellow]Summaries:[/bold yellow]")
    pprint(json.dumps(summaries, indent=4))

    console.print("\n[bold yellow]File Tree:[/bold yellow]")
    pprint(json.dumps(file_tree, indent=4))

    console.print("\n[bold yellow]Concatenated Data:[/bold yellow]")
    pprint(json.dumps(concatenated_data, indent=4))


15:53:53.355 | INFO    | prefect.engine - Created flow run 'ochre-sidewinder' for flow 'Document Processing Workflow'

15:53:53.411 | INFO    | prefect - Success callback set successfully.

15:53:53.413 | INFO    | Task run 'set_success_callback-659' - Finished in state Completed()

                                              Available Ollama Models                                              
                   ╷              ╷              ╷              ╷            ╷             ╷        ╷              
  Model Name       │ Modified At  │ Digest       │ Size (bytes) │ Param Size │ Quant Level │ Family │ Families     
 ══════════════════╪══════════════╪══════════════╪══════════════╪════════════╪═════════════╪════════╪═════════════ 
  smollm2:135m     │ 2024-12-18   │ 9077fe9d2ae… │    270898672 │    134.52M │ F16         │ llama  │ llama        
                   │ 11:49:06.80… │              │              │            │             │        │              
  llava:latest     │ 2024-12-07   │ 8dd30f6b0cb… │   4733363377 │         7B │ Q4_0        │ llama  │ llama, clip  
                   │ 23:46:35.56… │              │              │            │             │        │              
  moondream:latest │ 2024-12-07   │ 55fc3abd386… │   1738451197 │         1B │ Q4_0        │ phi2   │ phi2, clip   
                   │ 22:19:46.41… │              │              │            │             │        │              
  llama3.2:latest  │ 2024-12-07   │ a80c4f17acd… │   2019393189 │       3.2B │ Q4_K_M      │ llama  │ llama        
                   │ 22:18:36.84… │              │              │            │             │        │              
                   ╵              ╵              ╵              ╵            ╵             ╵        ╵

15:53:53.446 | INFO    | Task run 'list_ollama_models-a28' - Finished in state Completed()

...

('{\n'
 '    "files": [\n'
 '        {\n'
 '            "file_path": '
 '"/home/rakesh/Downloads/sample_data/four/dsflsdflj.txt",\n'
 '            "summary": "Introduction to Vertical Large Language Model (LLM) '
 'Agents and their applications in Software as a Service (SaaS) innovations, '
 'including the success story of Case Text\'s Co-Counsel."\n'
 '        },\n'
 '        {\n'
 '            "file_path": '
 '"/home/rakesh/Downloads/sample_data/four/random_file.txt",\n'
 '            "summary": "A test file containing important information, such '
 'as a bank account number."\n'
 '        },\n'
 '        {\n'
 '            "file_path": '
 '"/home/rakesh/Downloads/sample_data/four/shad.txt",\n'
 '            "summary": "Information about Next.js, including its features, '
 'technologies used, and recommended boilerplates for development."\n'
 '        }\n'
 '    ],\n'
 '    "usage": {\n'
 '        "completion_tokens": 139,\n'
 '        "prompt_tokens": 1436,\n'
 '        "total_token

File Tree:

'[]'


Concatenated Data:

'[]'


# Anthropic

In [27]:
# Example: Using Anthropic Haiku (via litellm)
# Ensure you have your ANTHROPIC_API_KEY set in the environment or .env.local
summaries_an = query_summaries(
    doc_dicts=doc_dicts,
    host="",         # Not needed if api_base is fully provided
    port="", 
    model="anthropic/claude-3-haiku-20240307",
    api_base="https://api.anthropic.com"  # For example, if litellm expects this pattern
)

pprint(summaries_an)

Calculated cost: 0.0061920000000000005
Summaries: >>>>>>>>>>> 
{'files': [{'file_path': '/home/rakesh/Downloads/sample_data/four/dsflsdflj.txt',
            'summary': 'This file discusses the emergence of Vertical Large '
                       'Language Model (LLM) Agents as a transformative '
                       'technology in the field of artificial intelligence. It '
                       "highlights the success story of Case Text's "
                       'Co-Counsel, an AI-driven legal assistant that '
                       "significantly increased the company's valuation. The "
                       'file explores why these vertical LLM agents are poised '
                       'to become billion-dollar opportunities and how '
                       'organizations can leverage this technology.'},
           {'file_path': '/home/rakesh/Downloads/sample_data/four/random_file.txt',
            'summary': 'This is a test file containing some important '
                    

In [34]:
pprint(summaries_an)

{'cost': 0.0061920000000000005,
 'files': [{'file_path': '/home/rakesh/Downloads/sample_data/four/dsflsdflj.txt',
            'summary': 'This file discusses the emergence of Vertical Large '
                       'Language Model (LLM) Agents as a transformative '
                       'technology in the field of artificial intelligence. It '
                       "highlights the success story of Case Text's "
                       'Co-Counsel, an AI-driven legal assistant that '
                       "significantly increased the company's valuation. The "
                       'file explores why these vertical LLM agents are poised '
                       'to become billion-dollar opportunities and how '
                       'organizations can leverage this technology.'},
           {'file_path': '/home/rakesh/Downloads/sample_data/four/random_file.txt',
            'summary': 'This is a test file containing some important '
                       'information, including a ba

In [28]:
tree_an = create_file_tree(
    summaries=summaries_an,
    host="",         # Not needed if api_base is fully provided
    port="", 
    model="anthropic/claude-3-haiku-20240307",
    api_base="https://api.anthropic.com"  # For example, if litellm expects this pattern
)

pprint(tree_an)


Calculated cost:[{'dst_path': '/home/rakesh/Documents/AI_Research/Vertical_LLM_Agents/co_counsel_case_study.txt',
  'src_path': '/home/rakesh/Downloads/sample_data/four/dsflsdflj.txt'},
 {'dst_path': '/home/rakesh/Documents/Sensitive_Information/bank_account_details.txt',
  'src_path': '/home/rakesh/Downloads/sample_data/four/random_file.txt'},
 {'dst_path': '/home/rakesh/Documents/Web_Development/NextJS_Boilerplates/overview.txt',
  'src_path': '/home/rakesh/Downloads/sample_data/four/shad.txt'}]
 0.003762


In [33]:
pprint(tree_an)

[{'dst_path': '/home/rakesh/Documents/AI_Research/Vertical_LLM_Agents/co_counsel_case_study.txt',
  'src_path': '/home/rakesh/Downloads/sample_data/four/dsflsdflj.txt'},
 {'dst_path': '/home/rakesh/Documents/Sensitive_Information/bank_account_details.txt',
  'src_path': '/home/rakesh/Downloads/sample_data/four/random_file.txt'},
 {'dst_path': '/home/rakesh/Documents/Web_Development/NextJS_Boilerplates/overview.txt',
  'src_path': '/home/rakesh/Downloads/sample_data/four/shad.txt'}]
